**Взято из источника** ([Источник](https://www.kaggle.com/aipi12/top-4-approach-with-randomforest))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Все для классификации

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV 

# Функция для построения графиков

In [ ]:
def plot_grid(data, fig_size, grid_size, plot_type, target = ''):
    fig = plt.figure(figsize = fig_size)
    if plot_type == 'histplot':
        for i, column_name in enumerate(data.select_dtypes(exclude = 'object').columns):
            fig.add_subplot(grid_size[0], grid_size[1], i + 1)
            plot = sns.histplot(data[column_name], kde = True, color = 'royalblue', stat = 'count')
    if plot_type == 'boxplot':
        for i, column_name in enumerate(data.select_dtypes(exclude = 'object').columns):
            fig.add_subplot(grid_size[0], grid_size[1], i + 1)
            plot = sns.boxplot(x = data[column_name], color = 'royalblue')
    if plot_type == 'countplot':
        target = data[target]
        for i, column_name in enumerate(data.drop(target.name, axis = 1).columns):
            fig.add_subplot(grid_size[0], grid_size[1], i + 1)
            plot = sns.countplot(x = data[column_name], hue = target, palette = 'Blues_r')
            plot.legend(loc = 'upper right', title = target.name)
    plt.tight_layout()

----

# plot_grid(train.drop('Survived', axis = 1), (16, 6), (2,3), 'histplot')
![](https://www.kaggleusercontent.com/kf/58549003/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pTm5qRVMWdaBMyxqo9D9iQ.6WYbgzS1Vg20okxQ2brNOVC2PikATdRvjEwtonhLV_zwkbFC-IKTp8o3WOgzt5LhuIE56hP7QtAXOeOM_HyaJ_RanV5rjvtE34dGA37kHmExZ1fRNQO8o975uop_JKMezNk8QToDK3vhfqj19ricVjKKlS6yANjBHvWCKjFt_z72XLO7JdesxXRN0GSU1scH4S1EXXoDUrZoT9DfF8r9TMXQnOymPCj79S6Tpn3iJSMi7fBHwq3UjeIgRQHai99mx0ZkJaUpQ1tmeu1IaAC3-T8gEJaezOPpHhLrwCjmRngxzPDLtjwknRIcrZGhP6HbGiYek2rwlfB9RxutBH9u2UZbjgMRjSc3Z7eBo2aXSLeL7dEYlxH7Nx1urUOT8MK5x4XLsonUmzVYJOHvs7Yuj15EqB_ogxNVDCDSuoG0VYqfMgLOJr8KtWC4kQ-8zXOdk-QOdVqMe2GFnedYXdYT-COoXHSjVb4eMCpflyg-gGMf99N9xDXqK7GlHYYEaPWMijP8eIfHg06mSh36Nu0W9FcLDuGqUV9ALhB0tjsz-j5LuRNO1yRz-eh8WC4VG5OQaF5NlAejxbG46MtzqjINrYxdEGynkmrdlJoOn_OXZxedaIZGuXXxpG_z4Lan39hRW36JLsDvujDT_RzFG9fUCt7ob5qNaDMkML1p3N-aoLM.jWfiBkipltsFTm-cIyVrUQ/__results___files/__results___26_0.png)

# plot_grid(train.drop('Survived', axis = 1), (16, 6), (2,3), 'histplot')
![](https://www.kaggleusercontent.com/kf/58549003/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pTm5qRVMWdaBMyxqo9D9iQ.6WYbgzS1Vg20okxQ2brNOVC2PikATdRvjEwtonhLV_zwkbFC-IKTp8o3WOgzt5LhuIE56hP7QtAXOeOM_HyaJ_RanV5rjvtE34dGA37kHmExZ1fRNQO8o975uop_JKMezNk8QToDK3vhfqj19ricVjKKlS6yANjBHvWCKjFt_z72XLO7JdesxXRN0GSU1scH4S1EXXoDUrZoT9DfF8r9TMXQnOymPCj79S6Tpn3iJSMi7fBHwq3UjeIgRQHai99mx0ZkJaUpQ1tmeu1IaAC3-T8gEJaezOPpHhLrwCjmRngxzPDLtjwknRIcrZGhP6HbGiYek2rwlfB9RxutBH9u2UZbjgMRjSc3Z7eBo2aXSLeL7dEYlxH7Nx1urUOT8MK5x4XLsonUmzVYJOHvs7Yuj15EqB_ogxNVDCDSuoG0VYqfMgLOJr8KtWC4kQ-8zXOdk-QOdVqMe2GFnedYXdYT-COoXHSjVb4eMCpflyg-gGMf99N9xDXqK7GlHYYEaPWMijP8eIfHg06mSh36Nu0W9FcLDuGqUV9ALhB0tjsz-j5LuRNO1yRz-eh8WC4VG5OQaF5NlAejxbG46MtzqjINrYxdEGynkmrdlJoOn_OXZxedaIZGuXXxpG_z4Lan39hRW36JLsDvujDT_RzFG9fUCt7ob5qNaDMkML1p3N-aoLM.jWfiBkipltsFTm-cIyVrUQ/__results___files/__results___28_0.png)

In [ ]:
def cor():
    plt.figure(figsize = (16, 6))
    sns.heatmap(train.corr(), 
                annot = True,
                fmt = '.2f',
                square = True,
                cmap = "Blues_r", 
                mask = np.triu(train.corr()))

![](https://www.kaggleusercontent.com/kf/58549003/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pTm5qRVMWdaBMyxqo9D9iQ.6WYbgzS1Vg20okxQ2brNOVC2PikATdRvjEwtonhLV_zwkbFC-IKTp8o3WOgzt5LhuIE56hP7QtAXOeOM_HyaJ_RanV5rjvtE34dGA37kHmExZ1fRNQO8o975uop_JKMezNk8QToDK3vhfqj19ricVjKKlS6yANjBHvWCKjFt_z72XLO7JdesxXRN0GSU1scH4S1EXXoDUrZoT9DfF8r9TMXQnOymPCj79S6Tpn3iJSMi7fBHwq3UjeIgRQHai99mx0ZkJaUpQ1tmeu1IaAC3-T8gEJaezOPpHhLrwCjmRngxzPDLtjwknRIcrZGhP6HbGiYek2rwlfB9RxutBH9u2UZbjgMRjSc3Z7eBo2aXSLeL7dEYlxH7Nx1urUOT8MK5x4XLsonUmzVYJOHvs7Yuj15EqB_ogxNVDCDSuoG0VYqfMgLOJr8KtWC4kQ-8zXOdk-QOdVqMe2GFnedYXdYT-COoXHSjVb4eMCpflyg-gGMf99N9xDXqK7GlHYYEaPWMijP8eIfHg06mSh36Nu0W9FcLDuGqUV9ALhB0tjsz-j5LuRNO1yRz-eh8WC4VG5OQaF5NlAejxbG46MtzqjINrYxdEGynkmrdlJoOn_OXZxedaIZGuXXxpG_z4Lan39hRW36JLsDvujDT_RzFG9fUCt7ob5qNaDMkML1p3N-aoLM.jWfiBkipltsFTm-cIyVrUQ/__results___files/__results___31_1.png)

# plot_grid(train.drop('Survived', axis = 1), (16, 6), (2,3), 'boxplot')
![](https://www.kaggleusercontent.com/kf/58549003/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pTm5qRVMWdaBMyxqo9D9iQ.6WYbgzS1Vg20okxQ2brNOVC2PikATdRvjEwtonhLV_zwkbFC-IKTp8o3WOgzt5LhuIE56hP7QtAXOeOM_HyaJ_RanV5rjvtE34dGA37kHmExZ1fRNQO8o975uop_JKMezNk8QToDK3vhfqj19ricVjKKlS6yANjBHvWCKjFt_z72XLO7JdesxXRN0GSU1scH4S1EXXoDUrZoT9DfF8r9TMXQnOymPCj79S6Tpn3iJSMi7fBHwq3UjeIgRQHai99mx0ZkJaUpQ1tmeu1IaAC3-T8gEJaezOPpHhLrwCjmRngxzPDLtjwknRIcrZGhP6HbGiYek2rwlfB9RxutBH9u2UZbjgMRjSc3Z7eBo2aXSLeL7dEYlxH7Nx1urUOT8MK5x4XLsonUmzVYJOHvs7Yuj15EqB_ogxNVDCDSuoG0VYqfMgLOJr8KtWC4kQ-8zXOdk-QOdVqMe2GFnedYXdYT-COoXHSjVb4eMCpflyg-gGMf99N9xDXqK7GlHYYEaPWMijP8eIfHg06mSh36Nu0W9FcLDuGqUV9ALhB0tjsz-j5LuRNO1yRz-eh8WC4VG5OQaF5NlAejxbG46MtzqjINrYxdEGynkmrdlJoOn_OXZxedaIZGuXXxpG_z4Lan39hRW36JLsDvujDT_RzFG9fUCt7ob5qNaDMkML1p3N-aoLM.jWfiBkipltsFTm-cIyVrUQ/__results___files/__results___32_0.png)

## plot_grid(pd.concat([train.select_dtypes(include = 'object').drop(['Name', 'Ticket', 'Cabin'], axis = 1), target], axis = 1), (16, 6), (2,1), 'countplot', 'Survived')
![](https://www.kaggleusercontent.com/kf/58549003/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pTm5qRVMWdaBMyxqo9D9iQ.6WYbgzS1Vg20okxQ2brNOVC2PikATdRvjEwtonhLV_zwkbFC-IKTp8o3WOgzt5LhuIE56hP7QtAXOeOM_HyaJ_RanV5rjvtE34dGA37kHmExZ1fRNQO8o975uop_JKMezNk8QToDK3vhfqj19ricVjKKlS6yANjBHvWCKjFt_z72XLO7JdesxXRN0GSU1scH4S1EXXoDUrZoT9DfF8r9TMXQnOymPCj79S6Tpn3iJSMi7fBHwq3UjeIgRQHai99mx0ZkJaUpQ1tmeu1IaAC3-T8gEJaezOPpHhLrwCjmRngxzPDLtjwknRIcrZGhP6HbGiYek2rwlfB9RxutBH9u2UZbjgMRjSc3Z7eBo2aXSLeL7dEYlxH7Nx1urUOT8MK5x4XLsonUmzVYJOHvs7Yuj15EqB_ogxNVDCDSuoG0VYqfMgLOJr8KtWC4kQ-8zXOdk-QOdVqMe2GFnedYXdYT-COoXHSjVb4eMCpflyg-gGMf99N9xDXqK7GlHYYEaPWMijP8eIfHg06mSh36Nu0W9FcLDuGqUV9ALhB0tjsz-j5LuRNO1yRz-eh8WC4VG5OQaF5NlAejxbG46MtzqjINrYxdEGynkmrdlJoOn_OXZxedaIZGuXXxpG_z4Lan39hRW36JLsDvujDT_RzFG9fUCt7ob5qNaDMkML1p3N-aoLM.jWfiBkipltsFTm-cIyVrUQ/__results___files/__results___36_0.png)

In [ ]:
def histsns():
    fig, axs = plt.subplots(2, 2, figsize = (16, 6))

    sns.histplot(train.Age, kde = True, color = 'black', ax = axs[0, 0])
    axs[0, 0].set_title('Ages in train with NaN values')
    sns.histplot(train_cleaning.Age, kde = True, color = 'royalblue', ax = axs[0, 1])
    axs[0, 1].set_title('Ages in train without NaN values')

    sns.histplot(test.Age, kde = True, color = 'black', ax = axs[1, 0])
    axs[1, 0].set_title('Ages in test with NaN values')
    sns.histplot(test_cleaning.Age, kde = True, color = 'royalblue', ax = axs[1, 1])
    axs[1, 1].set_title('Ages in test without NaN values')

    fig.tight_layout()

![](https://www.kaggleusercontent.com/kf/58549003/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pTm5qRVMWdaBMyxqo9D9iQ.6WYbgzS1Vg20okxQ2brNOVC2PikATdRvjEwtonhLV_zwkbFC-IKTp8o3WOgzt5LhuIE56hP7QtAXOeOM_HyaJ_RanV5rjvtE34dGA37kHmExZ1fRNQO8o975uop_JKMezNk8QToDK3vhfqj19ricVjKKlS6yANjBHvWCKjFt_z72XLO7JdesxXRN0GSU1scH4S1EXXoDUrZoT9DfF8r9TMXQnOymPCj79S6Tpn3iJSMi7fBHwq3UjeIgRQHai99mx0ZkJaUpQ1tmeu1IaAC3-T8gEJaezOPpHhLrwCjmRngxzPDLtjwknRIcrZGhP6HbGiYek2rwlfB9RxutBH9u2UZbjgMRjSc3Z7eBo2aXSLeL7dEYlxH7Nx1urUOT8MK5x4XLsonUmzVYJOHvs7Yuj15EqB_ogxNVDCDSuoG0VYqfMgLOJr8KtWC4kQ-8zXOdk-QOdVqMe2GFnedYXdYT-COoXHSjVb4eMCpflyg-gGMf99N9xDXqK7GlHYYEaPWMijP8eIfHg06mSh36Nu0W9FcLDuGqUV9ALhB0tjsz-j5LuRNO1yRz-eh8WC4VG5OQaF5NlAejxbG46MtzqjINrYxdEGynkmrdlJoOn_OXZxedaIZGuXXxpG_z4Lan39hRW36JLsDvujDT_RzFG9fUCt7ob5qNaDMkML1p3N-aoLM.jWfiBkipltsFTm-cIyVrUQ/__results___files/__results___47_0.png)

In [ ]:
def colorhist():
    plt.figure(figsize = (16, 6))
    plot = sns.countplot(x = train_cleaning.Cabin.loc[train_cleaning.Cabin != 'None'].str.split().apply(lambda x: len(x)), 
                         hue = (train_cleaning.SibSp + train_cleaning.Parch + 1))
    plot.set_title('Relationship between number of places in cabin and family size')
    plot.set_xlabel('Number of places in cabin')
    plot.set_ylabel('Count')
    plot.legend(loc = 'upper right', title = 'Family size')

![](https://www.kaggleusercontent.com/kf/58549003/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pTm5qRVMWdaBMyxqo9D9iQ.6WYbgzS1Vg20okxQ2brNOVC2PikATdRvjEwtonhLV_zwkbFC-IKTp8o3WOgzt5LhuIE56hP7QtAXOeOM_HyaJ_RanV5rjvtE34dGA37kHmExZ1fRNQO8o975uop_JKMezNk8QToDK3vhfqj19ricVjKKlS6yANjBHvWCKjFt_z72XLO7JdesxXRN0GSU1scH4S1EXXoDUrZoT9DfF8r9TMXQnOymPCj79S6Tpn3iJSMi7fBHwq3UjeIgRQHai99mx0ZkJaUpQ1tmeu1IaAC3-T8gEJaezOPpHhLrwCjmRngxzPDLtjwknRIcrZGhP6HbGiYek2rwlfB9RxutBH9u2UZbjgMRjSc3Z7eBo2aXSLeL7dEYlxH7Nx1urUOT8MK5x4XLsonUmzVYJOHvs7Yuj15EqB_ogxNVDCDSuoG0VYqfMgLOJr8KtWC4kQ-8zXOdk-QOdVqMe2GFnedYXdYT-COoXHSjVb4eMCpflyg-gGMf99N9xDXqK7GlHYYEaPWMijP8eIfHg06mSh36Nu0W9FcLDuGqUV9ALhB0tjsz-j5LuRNO1yRz-eh8WC4VG5OQaF5NlAejxbG46MtzqjINrYxdEGynkmrdlJoOn_OXZxedaIZGuXXxpG_z4Lan39hRW36JLsDvujDT_RzFG9fUCt7ob5qNaDMkML1p3N-aoLM.jWfiBkipltsFTm-cIyVrUQ/__results___files/__results___56_1.png)

In [ ]:
def hromohist():
    fig, axs = plt.subplots(1, 2, figsize = (16, 6))
    sns.histplot(hue = target_cleaned, x = train_cleaning.Age.loc[train_cleaning.Sex == 'male'], palette = {0 : 'black', 1 : 'royalblue'}, ax = axs[0])
    axs[0].set_title('Male Age distribution')
    sns.histplot(hue = target_cleaned, x = train_cleaning.Age.loc[train_cleaning.Sex == 'female'], palette = {0 : 'black', 1 : 'royalblue'}, ax = axs[1])
    axs[1].set_title('Female Age distribution')
    plt.tight_layout()

![](https://www.kaggleusercontent.com/kf/58549003/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pTm5qRVMWdaBMyxqo9D9iQ.6WYbgzS1Vg20okxQ2brNOVC2PikATdRvjEwtonhLV_zwkbFC-IKTp8o3WOgzt5LhuIE56hP7QtAXOeOM_HyaJ_RanV5rjvtE34dGA37kHmExZ1fRNQO8o975uop_JKMezNk8QToDK3vhfqj19ricVjKKlS6yANjBHvWCKjFt_z72XLO7JdesxXRN0GSU1scH4S1EXXoDUrZoT9DfF8r9TMXQnOymPCj79S6Tpn3iJSMi7fBHwq3UjeIgRQHai99mx0ZkJaUpQ1tmeu1IaAC3-T8gEJaezOPpHhLrwCjmRngxzPDLtjwknRIcrZGhP6HbGiYek2rwlfB9RxutBH9u2UZbjgMRjSc3Z7eBo2aXSLeL7dEYlxH7Nx1urUOT8MK5x4XLsonUmzVYJOHvs7Yuj15EqB_ogxNVDCDSuoG0VYqfMgLOJr8KtWC4kQ-8zXOdk-QOdVqMe2GFnedYXdYT-COoXHSjVb4eMCpflyg-gGMf99N9xDXqK7GlHYYEaPWMijP8eIfHg06mSh36Nu0W9FcLDuGqUV9ALhB0tjsz-j5LuRNO1yRz-eh8WC4VG5OQaF5NlAejxbG46MtzqjINrYxdEGynkmrdlJoOn_OXZxedaIZGuXXxpG_z4Lan39hRW36JLsDvujDT_RzFG9fUCt7ob5qNaDMkML1p3N-aoLM.jWfiBkipltsFTm-cIyVrUQ/__results___files/__results___61_0.png)

In [ ]:
def kdeplot():
    plt.figure(figsize = (16,6))
    plot = sns.kdeplot(hue = target_cleaned, x = train_cleaning.Age, palette = {0 : 'black', 1 : 'royalblue'}, fill = True)
    plot.set_title('Age distribution')

![](https://www.kaggleusercontent.com/kf/58549003/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..pTm5qRVMWdaBMyxqo9D9iQ.6WYbgzS1Vg20okxQ2brNOVC2PikATdRvjEwtonhLV_zwkbFC-IKTp8o3WOgzt5LhuIE56hP7QtAXOeOM_HyaJ_RanV5rjvtE34dGA37kHmExZ1fRNQO8o975uop_JKMezNk8QToDK3vhfqj19ricVjKKlS6yANjBHvWCKjFt_z72XLO7JdesxXRN0GSU1scH4S1EXXoDUrZoT9DfF8r9TMXQnOymPCj79S6Tpn3iJSMi7fBHwq3UjeIgRQHai99mx0ZkJaUpQ1tmeu1IaAC3-T8gEJaezOPpHhLrwCjmRngxzPDLtjwknRIcrZGhP6HbGiYek2rwlfB9RxutBH9u2UZbjgMRjSc3Z7eBo2aXSLeL7dEYlxH7Nx1urUOT8MK5x4XLsonUmzVYJOHvs7Yuj15EqB_ogxNVDCDSuoG0VYqfMgLOJr8KtWC4kQ-8zXOdk-QOdVqMe2GFnedYXdYT-COoXHSjVb4eMCpflyg-gGMf99N9xDXqK7GlHYYEaPWMijP8eIfHg06mSh36Nu0W9FcLDuGqUV9ALhB0tjsz-j5LuRNO1yRz-eh8WC4VG5OQaF5NlAejxbG46MtzqjINrYxdEGynkmrdlJoOn_OXZxedaIZGuXXxpG_z4Lan39hRW36JLsDvujDT_RzFG9fUCt7ob5qNaDMkML1p3N-aoLM.jWfiBkipltsFTm-cIyVrUQ/__results___files/__results___62_1.png)

# Для проверки моделей на точность 

In [ ]:
def pred():
    lr = LogisticRegression()
    dt = DecisionTreeClassifier(random_state = 1)
    rf = RandomForestClassifier(random_state = 1)
    svc = make_pipeline(StandardScaler(), SVC(probability = True))
    knn = make_pipeline(StandardScaler(), KNeighborsClassifier())


    estimators = [lr,
                  dt,
                  rf,
                  svc, 
                  knn,]

    labels = ['Log Regression',
              'Decision Tree',
              'Random Forest',
              'SVC', 
              'KNN',]

    results = test_estimators(X_train_full, y_train_full, estimators, labels, cv = 10)
    results.style.background_gradient(cmap = 'Blues')

![](https://imgur.com/ZKGy5gL.png)